In [23]:
#split train and test data
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/preprocessing_data.csv")

train, test = train_test_split(df, test_size=0.1)


In [24]:
train = pd.DataFrame(train, columns=["image", "emotion"])
train.to_csv(cwd + "/data_csv/train.csv")

train.groupby('emotion').count()

,image
emotion,
ANGER,73
DISGUST,6
HAPPINESS,2132
NEUTRAL,2035
SADNESS,35
SURPRISE,72


In [25]:
test = pd.DataFrame(test, columns=["image", "emotion"])
test.to_csv(cwd + "/data_csv/test.csv")

test.groupby('emotion').count()

,image
emotion,
ANGER,6
HAPPINESS,247
NEUTRAL,214
SADNESS,2
SURPRISE,15


In [26]:
'''
import cv2,matplotlib.pyplot as plt,dlib,imutils
from imutils import face_utils

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")

image=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")
# image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
rects = detector(gray, 1)

for rect in rects:
    pred=predictor(gray,rect)
    fig, ax1 = plt.subplots()

    ax1.imshow(image)
    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])
    
# del predictor
'''

'\nimport cv2,matplotlib.pyplot as plt,dlib,imutils\nfrom imutils import face_utils\n\ndetector = dlib.get_frontal_face_detector()\npredictor = dlib.shape_predictor("../input/dlib-68/shape_predictor_68_face_landmarks.dat")\n\nimage=plt.imread("../input/recognizing-faces-in-the-wild/train/F0002/MID1/P00009_face3.jpg")\n# image = imutils.resize(image, width=500)\ngray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\nrects = detector(gray, 1)\n\nfor rect in rects:\n    pred=predictor(gray,rect)\n    fig, ax1 = plt.subplots()\n\n    ax1.imshow(image)\n    ax1.scatter(face_utils.shape_to_np(pred)[:,0],face_utils.shape_to_np(pred)[:,1])\n    \n# del predictor\n'

In [27]:
'''
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

data = pd.read_csv(cwd + "/train_data.csv")

#independent columns
X = data.iloc[:,0:20]

#target column
y = data.iloc[:,-1]  

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores],axis=1)

#naming the dataframe columns
featureScores.columns = ['Specs','Score']

#print 10 best features
print(featureScores.nlargest(10,'Score'))
'''

'\nfrom sklearn.feature_selection import SelectKBest\nfrom sklearn.feature_selection import chi2\n\ndata = pd.read_csv(cwd + "/train_data.csv")\n\n#independent columns\nX = data.iloc[:,0:20]\n\n#target column\ny = data.iloc[:,-1]  \n\n#apply SelectKBest class to extract top 10 best features\nbestfeatures = SelectKBest(score_func=chi2, k=10)\nfit = bestfeatures.fit(X,y)\ndfscores = pd.DataFrame(fit.scores_)\ndfcolumns = pd.DataFrame(X.columns)\n\n#concat two dataframes for better visualization \nfeatureScores = pd.concat([dfcolumns, dfscores],axis=1)\n\n#naming the dataframe columns\nfeatureScores.columns = [\'Specs\',\'Score\']\n\n#print 10 best features\nprint(featureScores.nlargest(10,\'Score\'))\n'

In [28]:
'''
from sklearn.preprocessing import StandardScaler

label = train.head(1000)
data = train.head(1000)
data.shape

std = StandardScaler().fit_transform(data)
std.shape
'''

'\nfrom sklearn.preprocessing import StandardScaler\n\nlabel = train.head(1000)\ndata = train.head(1000)\ndata.shape\n\nstd = StandardScaler().fit_transform(data)\nstd.shape\n'

In [29]:
'''
import cv2, glob, random, math, numpy as np, dlib, itertools
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model

emotions = ["0","1","2","3","4","5","6"]#, "Happy", "Neutral", "Sad"] #Emotion list
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat") 

#clf = SVC(kernel='linear', probability=True, tol=1e-3)

clf = RandomForestClassifier(min_samples_leaf=3)

#clf = linear_model.LogisticRegression(C=1e5)


#n_estimators = 10
#clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel='linear', probability=True, class_weight='balanced'), max_samples=1.0 / n_estimators, n_estimators=n_estimators),n_jobs=-1)


def get_files(emotion):
    print("/content/drive/MyDrive/Colab Notebooks/images/%s/*" %emotion)
    files = glob.glob(r"/content/drive/MyDrive/Colab Notebooks/images/%s/*" %emotion)
    print(len(files))
   
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
   
    return training, prediction

def get_landmarks(image):
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
            
        xmean = np.mean(xlist) #Get the mean of both axes to determine centre of gravity
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist] #get distance between each point and the central point in both axes
        ycentral = [(y-ymean) for y in ylist]

        if xlist[26] == xlist[29]: #If x-coordinates of the set are the same, the angle is 0, catch to prevent 'divide by 0' error in function
            anglenose = 0
        else:
            anglenose = int(math.atan((ylist[26]-ylist[29])/(xlist[26]-xlist[29]))*180/math.pi)

        if anglenose < 0:
            anglenose += 90
        else:
            anglenose -= 90

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x)
            landmarks_vectorised.append(y)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            anglerelative = (math.atan((z-ymean)/(w-xmean))*180/math.pi) - anglenose
            landmarks_vectorised.append(dist)
            landmarks_vectorised.append(anglerelative)

    if len(detections) < 1: 
        landmarks_vectorised = "error"
    return landmarks_vectorised

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        for item in training:
           
            image = cv2.imread(item) #open image
            if image is not None:
              #print(image)
              #gray = cv2.cvtColor(np.float32(image), cv2.COLOR_RGB2GRAY)
              gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
              #gray = rgb2gray(image)

              clahe_image = clahe.apply(gray)
              landmarks_vectorised = get_gabor(clahe_image)
            #if landmarks_vectorised == "error":
                #pass
            #else:
              training_data.append(landmarks_vectorised) #append image array to training data list
              training_labels.append(emotions.index(emotion))
    
        for item in prediction:
         
            image = cv2.imread(item)
            if image is not None:
              #gray = cv2.cvtColor(np.float32(image), cv2.COLOR_RGB2GRAY)
              gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
              #gray = rgb2gray(image)

              clahe_image = clahe.apply(gray)
              landmarks_vectorised = get_gabor(clahe_image)
            #if landmarks_vectorised == "error":
                #pass
            #else:
              prediction_data.append(landmarks_vectorised)
              prediction_labels.append(emotions.index(emotion))

    return training_data, training_labels, prediction_data, prediction_labels   

accur_lin = [] # 10 random set generation
for i in range(0,10):
    print("Making sets %s" %i) #Make sets by random sampling 80/20%
    training_data, training_labels, prediction_data, prediction_labels = make_sets()

    npar_train = np.array(training_data) # numpy array
    npar_trainlabs = np.array(training_labels)
    
    print("training model %s" %i) #train model
    clf.fit(npar_train, training_labels)

    print("getting accuracies %s" %i)
    npar_pred = np.array(prediction_data)
    pred_lin = clf.score(npar_pred, prediction_labels)
    print ("Model: ", pred_lin)
    accur_lin.append(pred_lin) #Store accuracy in a list

print("Mean value accuracy in Model: %.3f" %np.mean(accur_lin))
'''

'\nimport cv2, glob, random, math, numpy as np, dlib, itertools\nfrom sklearn.svm import SVC\nfrom sklearn.ensemble import BaggingClassifier, RandomForestClassifier\nfrom sklearn.multiclass import OneVsRestClassifier\nfrom sklearn import linear_model\n\nemotions = ["0","1","2","3","4","5","6"]#, "Happy", "Neutral", "Sad"] #Emotion list\nclahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))\ndetector = dlib.get_frontal_face_detector()\npredictor = dlib.shape_predictor("/content/drive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat") \n\n#clf = SVC(kernel=\'linear\', probability=True, tol=1e-3)\n\nclf = RandomForestClassifier(min_samples_leaf=3)\n\n#clf = linear_model.LogisticRegression(C=1e5)\n\n\n#n_estimators = 10\n#clf = OneVsRestClassifier(BaggingClassifier(SVC(kernel=\'linear\', probability=True, class_weight=\'balanced\'), max_samples=1.0 / n_estimators, n_estimators=n_estimators),n_jobs=-1)\n\n\ndef get_files(emotion):\n    print("/content/drive/MyDrive/Colab N